# DB Connection

In [1]:
import cassandra 
print (cassandra.__version__)
import pandas
print(pandas.__version__)
import pandas as pd
import uuid

3.25.0
1.0.1


In [2]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.concurrent import execute_concurrent_with_args
# cloud_config1 = {
#         'secure_connect_bundle': '../secure-connect-capstone-project.zip'
# }
# auth_provider1 = PlainTextAuthProvider('JJvGOjuOPszNbfeoOwEinlHI', 'ZM-OR5KZAIMzqksmn01xobmZW4cNRUiIft0SnYuhW966f++ILhp+2Mq,AErDqKAF7Y0AF4ReSYLjTCgsLCqRJiL3+8uxJMoS-K6qMtM1vEmgi5IT5c62OvBJkhxBzq5T')
# cluster1 = Cluster(cloud=cloud_config1, auth_provider=auth_provider1)
# #connect to keyspace_name
# session1 = cluster1.connect('iac689')

cloud_config2= {
        'secure_connect_bundle': '../secure-connect-capstone-project2.zip'
}
auth_provider2 = PlainTextAuthProvider('OZTxZCgzatSukjKBoWpZimZs', 'cfph4pZMPt-REY_B0BTCZ-qxZfwgpIeZA2T69DIzccYlZ2I1+8gKKk6Ruk+G+70.CyFNoZymWef1Eo1KJZk,SjYbo9SgRZwY97JfG6J8uFLvNEJ8,nCjMbcGz.wgmlJC')
cluster2 = Cluster(cloud=cloud_config2, auth_provider=auth_provider2)
#connect to keyspace_name
session2 = cluster2.connect('iac689')

In [3]:
query = "SELECT * FROM system_schema.keyspaces;"
df = pd.DataFrame(list(session2.execute(query)))
df

,keyspace_name,durable_writes,graph_engine,replication
0,iac689,True,None,"[class, us-east1]"
1,system_auth,True,None,"[class, us-east1]"
2,system_schema,False,None,[class]
3,data_endpoint_auth,True,None,"[class, us-east1]"
4,datastax_sla,True,None,"[class, us-east1]"
5,system,False,None,[class]
6,system_traces,True,None,"[class, us-east1]"


In [4]:
query = "SELECT * FROM system_schema.tables WHERE keyspace_name='iac689';"
df = pd.DataFrame(list(session2.execute(query)))
df

,keyspace_name,table_name,additional_write_policy,bloom_filter_fp_chance,caching,cdc,comment,compaction,compression,crc_check_chance,...,flags,gc_grace_seconds,id,max_index_interval,memtable_flush_period_in_ms,min_index_interval,nodesync,read_repair,read_repair_chance,speculative_retry
0,iac689,data_2,99PERCENTILE,0.01,"[keys, rows_per_partition]",None,,[class],"[chunk_length_in_kb, class]",1.0,...,(compound),864000,a68727d0-7f1c-11ec-bb17-45ae5e7b1bda,2048,0,128,None,BLOCKING,0.0,99PERCENTILE
1,iac689,kmeans,99PERCENTILE,0.01,"[keys, rows_per_partition]",None,,[class],"[chunk_length_in_kb, class]",1.0,...,(compound),864000,95ef9290-8944-11ec-bb17-45ae5e7b1bda,2048,0,128,None,BLOCKING,0.0,99PERCENTILE
2,iac689,newdata,99PERCENTILE,0.01,"[keys, rows_per_partition]",None,,[class],"[chunk_length_in_kb, class]",1.0,...,(compound),864000,3571f450-8dcf-11ec-a016-ef0b22327c1e,2048,0,128,None,BLOCKING,0.0,99PERCENTILE
3,iac689,test_1,99PERCENTILE,0.01,"[keys, rows_per_partition]",None,,[class],"[chunk_length_in_kb, class]",1.0,...,(compound),864000,8fccabc0-8df2-11ec-a016-ef0b22327c1e,2048,0,128,None,BLOCKING,0.0,99PERCENTILE


In [5]:
query = "SELECT * FROM system_schema.columns WHERE keyspace_name='iac689' and table_name = 'data_1';"
df = pd.DataFrame(list(session2.execute(query)))
df

""


# DB_Insert

In [6]:
import pandas as pd
import glob

In [7]:
path_list = glob.glob('../../cap_data/*.csv')
df_list = []
for i in path_list:
    df_list.append(pd.read_csv(i))

In [8]:
for i in range(len(df_list)):
    if (i < 5):
        df_list[i]['time'] = pd.to_datetime(df_list[i]['time']).dt.date
        df_list[i]['truck_name'] = df_list[i]['truck_name'].str.replace(r"[a-zA-Z|-]","")
        df_list[i]['truck_name'] = 'abc-'+df_list[i]['truck_id'].astype(str)
    else:
        df_list[i]['start_time'] = pd.to_datetime(df_list[i]['start_time']).dt.date
        df_list[i]['truck_name'] = df_list[i]['truck_name'].str.replace(r"[a-zA-Z|-]","")
        df_list[i]['truck_name'] = 'abc-'+df_list[i]['truck_id'].astype(str)
       

In [172]:

#Insert for the last dataframe into a different table (kind of slow)
# query = "INSERT INTO data_1 (uid, active, reading_id, start_mileage, start_time, truck_id, truck_name, type) VALUES (?,?,?,?,?,?,?,?)"
# prepared = session1.prepare(query)
# for i in df.itertuples():
#     print(i.truck_id)
#     session1.execute(prepared, (uuid.uuid1(), i.active, i.reading_id, i.start_mileage, i.start_time, i.truck_id, i.truck_name, i.not_active))

In [38]:
df_engine_speed = df_list[0].copy()
df_altitude = df_list[1].copy().dropna()
df_inclination = df_list[2].copy()
df_wheel_based_speed = pd.concat([df_list[3],df_list[4]])

In [39]:
inclination_hist = [-20, -18,-15,-13, -11.5,-10.5,-9.5,-8.5,-7.5,-6.5,-5.5,-4.5,-3.5,-2.5,-1.5,-0.5,0.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5,8.5,9.5,10.5,11.5,13.0,15.0,18.0,20.0]
altitude_hist = [i for i in range(500, 7000, 500)]
vehicle_speed_hist = [0,5,12,19,26,33,40,46,53,60,68,75,82,89,96,103,110,116,123,130]
engine_speed_hist = [i for i in range(0,12500, 500)]

In [40]:
def helper(df, bin_val:list):
#     df= df.pivot_table(index=['time', 'truck_id','type'], columns='bin_num',
#                      values='value').reset_index()
    df = df[df.bin_num !='start_mileage']
    num = len(df['bin_num'].unique())
    bin_val = bin_val[:num]
    print(len(bin_val),num)
    new_val = []
    for i in df['bin_num']:
        n = int(i.split('_')[-1])
        new_val.append(bin_val[n])
    df['new_val'] = new_val
    df['final_val'] = (df['value']*df['new_val'])/375
    return df

In [41]:
df_engine_speed = helper(df_engine_speed, engine_speed_hist)
df_inclination = helper(df_inclination, inclination_hist)
df_wheel_based_speed = helper(df_wheel_based_speed, vehicle_speed_hist)

24 24


/Users/vathanahim/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/Users/vathanahim/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


31 31
19 19


In [42]:
df_altitude = df_altitude.rename(columns={'value':'final_val'})
df_altitude

,time,truck_name,truck_id,type,reading_id,bin_num,final_val
0,2021-11-17,abc-242,242,Time(hours) at different Altitudes,86021,col_0,0.000000
1,2021-11-17,abc-242,242,Distance(miles) at different Altitudes,86021,col_0,0.000000
2,2021-11-17,abc-242,242,Time(hours) at different Altitudes,86021,col_0,0.000000
3,2021-11-17,abc-242,242,Distance(miles) at different Altitudes,86021,col_0,0.000000
4,2021-01-04,abc-294,294,Time(hours) at different Altitudes,86022,col_0,0.772778
...,...,...,...,...,...,...,...
95467,2021-02-15,abc-277,277,Distance(miles) at different Altitudes,130151,col_0,0.000000
95468,2021-02-17,abc-277,277,Time(hours) at different Altitudes,130152,col_0,0.000000
95469,2021-02-17,abc-277,277,Distance(miles) at different Altitudes,130152,col_0,0.000000
95470,2021-02-17,abc-277,277,Time(hours) at different Altitudes,130152,col_0,0.000000


In [46]:
df_list = [df_engine_speed, df_altitude, df_inclination, df_wheel_based_speed]
df = pd.concat(df_list)


In [18]:
# df = df.pivot_table(index=['time', 'truck_id'], columns='type',
#                      values='final_val',aggfunc='mean').reset_index()
# df.columns

df = pd.concat(df_list)
df = df[df['bin_num']!='start_mileage']
df = df.pivot_table(index=['time', 'truck_id'], columns='type',
                     values='value').reset_index()
df.columns

Index(['time', 'truck_id', 'Distance(miles) at WheelBasedVehicleSpeed(mph)',
       'Distance(miles) at different Altitudes',
       'Distance(miles) at different Engine Speeds (rpms)',
       'Distance(miles) at different RoadInclination(%)',
       'Time(hours) at WheelBasedVehicleSpeed(mph)',
       'Time(hours) at different Altitudes',
       'Time(hours) at different Engine Speeds (rpms)',
       'Time(hours) at different RoadInclination(%)'],
      dtype='object', name='type')

In [19]:
df

type,time,truck_id,Distance(miles) at WheelBasedVehicleSpeed(mph),Distance(miles) at different Altitudes,Distance(miles) at different Engine Speeds (rpms),Distance(miles) at different RoadInclination(%),Time(hours) at WheelBasedVehicleSpeed(mph),Time(hours) at different Altitudes,Time(hours) at different Engine Speeds (rpms),Time(hours) at different RoadInclination(%)
0,2020-12-28,31,194.736842,12497.192308,154.166667,119.354839,0.096316,0.139626,0.076250,0.059032
1,2020-12-29,31,235.000000,11147.807692,186.041667,144.032258,0.096367,0.137062,0.076291,0.059064
2,2020-12-30,31,374.210526,9206.884615,296.250000,229.354839,0.086779,0.120449,0.068700,0.053187
3,2020-12-31,31,242.017544,11429.264103,191.597222,148.333333,0.079440,0.113120,0.062890,0.048689
4,2021-01-01,19,2735.236842,277766.989231,2165.395833,1676.436129,0.037107,0.054212,0.029376,0.022743
...,...,...,...,...,...,...,...,...,...,...
6115,2022-02-09,66,2923.505848,NaN,2314.446759,1791.829749,0.033013,NaN,0.026136,0.020234
6116,2022-02-09,69,20.921053,29367.838462,16.562500,12.822581,0.006637,0.009701,0.005255,0.004068
6117,2022-02-09,89,3589.094737,400709.184615,2841.375000,2199.774194,0.042406,0.061966,0.033572,0.025991
6118,2022-02-09,93,8434.473684,69079.307692,6677.291667,5169.516129,0.210541,0.303291,0.166678,0.129041


In [20]:
# df['Distance(miles) at WheelBasedVehicleSpeed(mph)'] = abs(df['Distance(miles) at WheelBasedVehicleSpeed(mph)'])
# df['Distance(miles) at different Engine Speeds (rpms)'] = abs(df['Distance(miles) at different Engine Speeds (rpms)'])
df.describe()

type,truck_id,Distance(miles) at WheelBasedVehicleSpeed(mph),Distance(miles) at different Altitudes,Distance(miles) at different Engine Speeds (rpms),Distance(miles) at different RoadInclination(%),Time(hours) at WheelBasedVehicleSpeed(mph),Time(hours) at different Altitudes,Time(hours) at different Engine Speeds (rpms),Time(hours) at different RoadInclination(%)
count,6120.000000,6.118000e+03,5.655000e+03,6.120000e+03,5.174000e+03,6118.000000,5655.000000,6120.000000,5174.000000
mean,53.685621,1.077430e+04,2.883362e+05,8.527773e+03,7.631794e+03,0.083101,0.124337,0.065780,0.051630
std,29.245279,3.692188e+05,4.104046e+05,2.922507e+05,2.460604e+05,0.073808,0.102592,0.058445,0.040649
min,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,-1.843548e+02,0.000044,0.000043,0.000035,0.000027
25%,29.000000,6.056374e+02,2.147235e+04,4.759635e+02,5.742724e+02,0.023639,0.035183,0.018697,0.016246
50%,50.000000,2.836177e+03,9.705261e+04,2.243302e+03,2.202659e+03,0.061280,0.096553,0.048454,0.039336
75%,78.000000,6.570602e+03,4.082279e+05,5.197353e+03,4.582401e+03,0.135297,0.202172,0.107068,0.085152
max,97.000000,2.583508e+07,3.971701e+06,2.045278e+07,1.583440e+07,1.994814,1.041669,1.579233,0.129077


In [21]:
df['uuid'] = df.index.to_series().map(lambda x: uuid.uuid4())
df

type,time,truck_id,Distance(miles) at WheelBasedVehicleSpeed(mph),Distance(miles) at different Altitudes,Distance(miles) at different Engine Speeds (rpms),Distance(miles) at different RoadInclination(%),Time(hours) at WheelBasedVehicleSpeed(mph),Time(hours) at different Altitudes,Time(hours) at different Engine Speeds (rpms),Time(hours) at different RoadInclination(%),uuid
0,2020-12-28,31,194.736842,12497.192308,154.166667,119.354839,0.096316,0.139626,0.076250,0.059032,d2996d12-fa36-43a4-bf1f-c1706bbbaf92
1,2020-12-29,31,235.000000,11147.807692,186.041667,144.032258,0.096367,0.137062,0.076291,0.059064,f04b8c59-e76f-43a0-b084-4799f70c1c03
2,2020-12-30,31,374.210526,9206.884615,296.250000,229.354839,0.086779,0.120449,0.068700,0.053187,13ff4dbd-96b1-40c1-a9f6-e8c3b57b96b7
3,2020-12-31,31,242.017544,11429.264103,191.597222,148.333333,0.079440,0.113120,0.062890,0.048689,05ad19f7-114b-4319-8be9-d4509ed67a47
4,2021-01-01,19,2735.236842,277766.989231,2165.395833,1676.436129,0.037107,0.054212,0.029376,0.022743,1f002e33-19d1-4f87-98fa-a832df38ad16
...,...,...,...,...,...,...,...,...,...,...,...
6115,2022-02-09,66,2923.505848,NaN,2314.446759,1791.829749,0.033013,NaN,0.026136,0.020234,04f0ebb3-1efb-4b88-b865-011087f4e593
6116,2022-02-09,69,20.921053,29367.838462,16.562500,12.822581,0.006637,0.009701,0.005255,0.004068,2b77aa53-b5f6-4543-99cc-ed3c2fdf4014
6117,2022-02-09,89,3589.094737,400709.184615,2841.375000,2199.774194,0.042406,0.061966,0.033572,0.025991,888a2a7a-f1ed-4413-bb89-bfd606f8e87c
6118,2022-02-09,93,8434.473684,69079.307692,6677.291667,5169.516129,0.210541,0.303291,0.166678,0.129041,c905563a-7831-4347-9277-7748759a0d8e


In [51]:
# df['Distance(miles) at WheelBasedVehicleSpeed(mph)'] = abs(df['Distance(miles) at WheelBasedVehicleSpeed(mph)'])
# df['Distance(miles) at different Engine Speeds (rpms)'] = abs(df['Distance(miles) at different Engine Speeds (rpms)'])
# df['Time(hours) at different Engine Speeds (rpms)']= abs((df['Time(hours) at different Engine Speeds (rpms)']-df['Time(hours) at different Engine Speeds (rpms)'].min())/(df['Time(hours) at different Engine Speeds (rpms)'].max()-df['Time(hours) at different Engine Speeds (rpms)'].min()))
# df['Time(hours) at different RoadInclination(%)'] = abs(df['Time(hours) at different RoadInclination(%)'])

In [22]:
l = []
for i in df[['uuid','time','Distance(miles) at different Altitudes','Distance(miles) at different Engine Speeds (rpms)','Distance(miles) at different RoadInclination(%)', 'Distance(miles) at WheelBasedVehicleSpeed(mph)','Time(hours) at different Altitudes', 'Time(hours) at different Engine Speeds (rpms)','Time(hours) at different RoadInclination(%)','Time(hours) at WheelBasedVehicleSpeed(mph)','truck_id']].columns:
    l.append(df[i].tolist())



In [23]:
query = """insert into newdata (ui , datel , distance_at_diff_altitude , distance_at_diff_engine_speeds , distance_at_diff_roadinclination , distance_at_wheelbased_vehicle_speed , time_at_diff_altitude , time_at_diff_engine_speeds , time_at_diff_roadinclination , time_at_wheelbased_vehicle_speed , truck_id)
values (?,?,?,?,?,?,?,?,?,?,?);"""
prepared = session2.prepare(query)
execute_concurrent_with_args(session2, prepared, zip(l[0],l[1],l[2],l[3],l[4],l[5],l[6],l[7],l[8],l[9],l[10]))

[ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7fb3c37fe910>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7fb3c37f3290>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7fb3c37d5d50>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7fb3c37f3050>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7fb3c8aca2d0>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7fb3c8acab90>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7fb3ab106fd0>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7fb3c37f3dd0>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7fb3c37e8b50>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object a

In [45]:
query = "select * from iac689.data_1;"
df = pd.DataFrame(list(session1.execute(query)))
df

,uid,active,reading_id,start_mileage,start_time,truck_id,truck_name,type
0,1b644c6e-7f03-11ec-99b7-acde48001122,5400.000000,103995,15906600.0,2021-02-19,205,abc-205,Distance(miles) with CC active
1,1ba064e2-7f03-11ec-99b7-acde48001122,7340.000000,103997,15912000.0,2021-02-19,205,abc-205,Distance(miles) with CC active
2,d847cb7c-7f02-11ec-99b7-acde48001122,11815.000000,103358,12433600.0,2021-01-13,205,abc-205,Distance(miles) with CC active
3,6a9d1424-7f02-11ec-99b7-acde48001122,0.021667,99698,17789500.0,2021-01-11,284,abc-284,Time(hours) with CC Active
4,f6d2f940-7f02-11ec-99b7-acde48001122,0.138056,103646,14102000.0,2021-02-01,205,abc-205,Time(hours) with CC Active
...,...,...,...,...,...,...,...,...
8525,eb7bfd62-7f02-11ec-99b7-acde48001122,0.000000,103539,13469200.0,2021-01-25,205,abc-205,Distance(miles) with CC active
8526,fe85cb18-7f02-11ec-99b7-acde48001122,0.115278,103720,14719100.0,2021-02-05,205,abc-205,Time(hours) with CC Active
8527,17da6c18-7f03-11ec-99b7-acde48001122,0.254722,103961,15560800.0,2021-02-16,205,abc-205,Time(hours) with CC Active
8528,5300048e-7f02-11ec-99b7-acde48001122,0.130556,86044,8068540.0,2021-02-15,294,abc-294,Time(hours) with CC Active


In [44]:
query = "select * from iac689.data_2;"
df = pd.DataFrame(list(session2.execute(query)))
df

,ui,datel,distance_at_diff_altitude,distance_at_diff_engine_speeds,distance_at_diff_roadinclination,distance_at_wheelbased_vehicle_speed,time_at_diff_altitude,time_at_diff_engine_speeds,time_at_diff_roadinclination,time_at_wheelbased_vehicle_speed,truck_id
0,1c9a40f7-28bd-448b-837f-7e6f26867cee,2021-01-12,2.468247e+05,7.139896e+05,5.578044e+05,9.648508e+05,0.318268,7.137108e+05,5.575866e+05,9.644741e+05,284
1,eed85ca4-db7c-42a3-b4f6-46660ac6f5ab,2021-10-01,0.000000e+00,1.087964e+07,8.499720e+06,1.470222e+07,0.000000,1.087643e+07,8.497214e+06,1.469788e+07,296
2,69b49f87-6525-4203-8168-a406da377443,2021-11-18,NaN,1.010224e+07,7.892374e+06,1.365167e+07,NaN,1.010193e+07,7.892130e+06,1.365125e+07,282
3,375516c0-521f-4411-b83a-74f84ab454be,2021-05-12,5.921890e+05,5.071094e+05,NaN,6.852830e+05,1.227917,5.051501e+05,NaN,6.826352e+05,294
4,3f15f13f-62e0-435a-910b-317c3fda47ac,2021-01-25,NaN,2.643425e+07,2.065176e+07,3.572196e+07,NaN,2.643222e+07,2.065017e+07,3.571922e+07,279
...,...,...,...,...,...,...,...,...,...,...,...
4542,2d46a503-3581-40eb-9ad9-124e7a362185,2021-07-27,5.230573e+05,8.439053e+05,NaN,1.140412e+06,0.531806,8.434997e+05,NaN,1.139864e+06,205
4543,9932ae23-5a33-42f1-9a87-ec803a18c325,2021-08-06,8.052310e+05,8.942562e+05,NaN,1.208454e+06,4.340280,8.896362e+05,NaN,1.202211e+06,294
4544,c0b4c700-96ab-438e-a272-e45ba97a3974,2021-06-03,2.373810e+06,2.268493e+05,NaN,3.065532e+05,2.066203,2.259348e+05,NaN,3.053172e+05,292
4545,e29ed179-f997-429f-b592-8158bd7b2030,2021-10-11,NaN,5.238979e+06,4.092952e+06,7.079684e+06,NaN,5.232000e+06,4.087500e+06,7.070270e+06,208


(ui uuid primary key, datel date, distance_at_diff_altitude float, distance_at_diff_engine_speeds float, distance_at_diff_roadinclination float, distance_at_wheelbased_vehicle_speed float, time_at_diff_altitude float, time_at_diff_engine_speeds float, time_at_diff_roadinclination float, time_at_wheelbased_vehicle_speed float, truck_id int)